In [1]:
import os
from dotenv import load_dotenv
from autogen import AssistantAgent, UserProxyAgent

In [2]:
load_dotenv()

True

In [5]:


llm_config = {
    "model": "gpt-4", 
    "api_key": os.environ["OPENAI_API_KEY"],
    "temperature": 0
}

assistant = AssistantAgent(
    name="assistant", 
    llm_config=llm_config,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    system_message="You are a smart chatbot, reply TERMINATE if you think you have answered user's question or if the user is sending empty message."
)
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    system_message="You are going to ask question one time only, reply TERMINATE if you think your question has a reaction",
    is_termination_msg=lambda content: "TERMINATE" in content["content"]
    
)

# Start the chat
user_proxy.initiate_chat(
    assistant,
    message="My name is bob",
)

user_proxy (to assistant):

My name is bob

--------------------------------------------------------------------------------
assistant (to user_proxy):

Nice to meet you, Bob! How can I assist you today?

--------------------------------------------------------------------------------
user_proxy (to assistant):



--------------------------------------------------------------------------------
assistant (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'My name is bob', 'role': 'assistant'}, {'content': 'Nice to meet you, Bob! How can I assist you today?', 'role': 'user'}, {'content': '', 'role': 'assistant'}, {'content': 'TERMINATE', 'role': 'user'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0.00426, 'gpt-4-0613': {'cost': 0.00426, 'prompt_tokens': 108, 'completion_tokens': 17, 'total_tokens': 125}}, 'usage_excluding_cached_inference': {'total_cost': 0.00426, 'gpt-4-0613': {'cost': 0.00426, 'prompt_tokens': 108, 'completion_tokens': 17, 'total_tokens': 125}}}, human_input=[])

In [6]:
user_proxy.initiate_chat(
    assistant,
    message="What is my name?",
)

user_proxy (to assistant):

What is my name?

--------------------------------------------------------------------------------
assistant (to user_proxy):

As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'What is my name?', 'role': 'assistant'}, {'content': "As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. TERMINATE", 'role': 'user'}], summary="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. ", cost={'usage_including_cached_inference': {'total_cost': 0.00798, 'gpt-4-0613': {'cost': 0.00798, 'prompt_tokens': 152, 'completion_tokens': 57, 'total_tokens': 209}}, 'usage_excluding_cached_inference': {'total_cost': 0.00798, 'gpt-4-0613': {'cost': 0.00798, 'prompt_tokens': 152, 'completion_tokens': 57, 'total_tokens': 209}}}, human_input=[])

In [ ]:
# for message in history:
#     user_proxy.send(message, assistant, request_reply=False) 

In [ ]:

# class ResumableGroupChatManager(GroupChatManager):
#     groupchat: GroupChat

#     def __init__(self, groupchat, history, **kwargs):
#         self.groupchat = groupchat
#         if history:
#             self.groupchat.messages = history

#         super().__init__(groupchat, **kwargs)

#         if history:
#             self.restore_from_history(history)

#     def restore_from_history(self, history) -> None:
#         for message in history:
#             # broadcast the message to all agents except the speaker.  This idea is the same way GroupChat is implemented in AutoGen for new messages, this method simply allows us to replay old messages first.
#             for agent in self.groupchat.agents:
#                 if agent != self:
#                     self.send(message, agent, request_reply=False, silent=True)
